In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv ('dataset.csv')
df_target = df['humor']
df_data = df.copy()
df_data.drop(columns='humor')

df_target.head()

0    False
1    False
2     True
3    False
4    False
Name: humor, dtype: bool

In [2]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df_target = label_encoder.fit_transform(df_target)
df_target

array([0, 0, 1, ..., 1, 0, 1])

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

#At the moment I am not able to apply the vectorizer on the whole dataset. To many records & less available RAM
#Apply TfidfVectorizer. The min_df parameter removes all tokens with occurence less then the given percentage. (Pruning)
#I am not sure what the norm parameter does! In the exercise it's sometimes used, sometimes not 

#tfidfVectorizer = TfidfVectorizer(use_idf=False, norm='l1',min_df=in_df=0.00009)

#Okay, let's go again with removing some stopwords
tfidfVectorizer = TfidfVectorizer(use_idf=False, min_df=0.00009)


matrix = tfidfVectorizer.fit_transform(df_data['text'])

df_data_tfidfVectorized = pd.DataFrame(matrix.toarray(), columns=tfidfVectorizer.get_feature_names())
display(df_data_tfidfVectorized)

,00,000,0k,10,100,1000,101,10th,11,12,...,zimmerman,zinke,zipper,zoe,zombie,zombies,zone,zones,zoo,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#kernel die when running this cell

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data_tfidfVectorized, df_target, test_size=0.2, random_state=42)

random_forest = RandomForestClassifier(max_depth=3, random_state=0)

random_forest.fit(df_data_train, df_target_train)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

df_prediction = random_forest.predict(df_data_test)

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
#print("Confusion Matrix:")
cnf_matrix = confusion_matrix(df_target, df_prediction)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=label_encoder.classes_, title='Random Forest Classifier')

print("Accuracy: {}".format(accuracy_score(df_target_test, df_prediction)))